<h1 align="center">Perceptron</h1>

## Import Libraries

In [347]:
import numpy as np
import pandas as pd

## Utility Functions

In [348]:
def linear(x):
    return x


def rectified_linear(x: np.ndarray):
    return np.maximum(0, x)


def sigmoid(x: np.ndarray):
    return 1 / (1 + np.exp(-x))

## The Perceptron Algorithm

In [349]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = rectified_linear

        self.weights = None
        self.bias = None

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        n_samples, n_features = X.shape

        # Initialize parameters
        self.weights = np.random.rand(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                y_predicted = self.predict(x_i)

                # Update weights and bias using delta rule
                update = self.lr * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X: np.ndarray) -> np.ndarray:
        net = np.dot(X, self.weights) + self.bias
        return self.activation_func(net)

In [350]:
model = Perceptron(learning_rate=0.069, n_iters=1000)

## Products Prediction

### Initial and clean data

In [351]:
prod_train = pd.read_csv("data/prod-train.csv")

In [352]:
type_mappings = {"Chair": 0, "Table": 1, "Bed": 2}

In [353]:
# Feature selection
features = ["Width", "Height"]

# Slipt data into X and Y
X_train = prod_train[features].values
Y_train = prod_train["Type"].map(type_mappings).values

### Training the Model 

In [354]:
model.fit(X_train, Y_train)

In [355]:
model.bias, model.weights

(-0.35263286154652096, array([1.26975217, 0.24323014]))

In [356]:
predictions = np.round(model.predict(X_train))

In [357]:
pd.DataFrame(
    {
        "Width": prod_train["Width"],
        "Height": prod_train["Height"],
        "Type": prod_train["Type"],
        "Predicted": np.where(
            predictions == 0, "Chair", np.where(predictions == 1, "Table", "Bed")
        ),
    }
)

,Width,Height,Type,Predicted
0,0.30,0.30,Chair,Chair
1,0.35,0.40,Chair,Chair
2,0.40,0.50,Chair,Chair
3,0.80,0.75,Table,Table
4,0.90,0.70,Table,Table
5,1.00,0.80,Table,Table
6,1.20,0.40,Bed,Table
7,1.60,0.50,Bed,Bed
8,2.00,0.50,Bed,Bed


### Testing the Model

In [358]:
prod_test = pd.read_csv("data/prod-test.csv")

In [359]:
X_test = prod_test[features].values

In [360]:
predictions = np.round(model.predict(X_test))

In [361]:
pd.DataFrame(
    {"Width": prod_test["Width"], "Height": prod_test["Height"], "Type": predictions}
).replace({0: "Chair", 1: "Table", 2: "Bed"})

,Width,Height,Type
0,0.35,0.35,Chair
1,0.85,0.70,Table
2,1.40,0.45,Bed


## And Gate Prediction

In [362]:
ag_train = pd.read_csv("data/and-gate.csv")

In [363]:
model.fit(ag_train[["B", "A"]].values, ag_train["Y"].values)

In [364]:
predictions = np.round(model.predict(ag_train[["B", "A"]].values))

In [365]:
pd.DataFrame(
    {
        "B": ag_train["B"],
        "A": ag_train["A"],
        "Y": ag_train["Y"],
        "Predicted": predictions,
    }
)

,B,A,Y,Predicted
0,0,0,0,0.0
1,0,1,0,0.0
2,1,0,0,0.0
3,1,1,1,1.0
